In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [34]:
learning_rate = 0.001
training_iterations = 200000
batch_size = 128
display_size = 10
n_input = 784
n_classes = 10
dropout = 0.75

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None,n_classes])
keep_probability = tf.placeholder(tf.float32)

In [35]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x,ksize = [1,k,k,1], strides = [1, k, k, 1], padding='SAME')

def convNet(x, weights, bias, dropout):
    x = tf.reshape(x, shape=[-1,28,28,1])
    #Convolution Layer 1
    conv1 = conv2d(x, weights['w1'], bias['b1'])
    conv1 = maxpool2d(conv1, k=2)
    #Convolution Layer 2
    conv2 = conv2d(conv1, weights['w2'], bias['b2'])
    conv2 = maxpool2d(conv2, k=2)
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), bias['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), bias['out'])
    return out
    

In [36]:
weights = {
    'w1': tf.Variable(tf.random_normal([5,5,1,32])),
    'w2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out':tf.Variable(tf.random_normal([1024, n_classes]))
}
bias = {
    'b1':tf.Variable(tf.random_normal([32])),
    'b2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

prediction = convNet(x, weights, bias, keep_probability)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction, y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [37]:
#Evaluation
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [38]:
model = tf.initialize_all_variables()
with tf.Session() as session:
    session.run(model)
    step = 1
    while step * batch_size < training_iterations:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        session.run(optimizer, feed_dict = {x: batch_x, y: batch_y, keep_probability: dropout})
        if step % display_size == 0:
            loss, acc = session.run([cost, accuracy], feed_dict = {x: batch_x, y: batch_y, keep_probability : 1.})
            print "Iter:" + str(step*batch_size) + ",Minibatch loss" + "{:.6f}".format(loss) + \
            ",Training_accuracy:" + "{:.5f}".format(acc)
        step+=1
        
    print "Optimization Finished"  
    print "Testing Accuracy:", \
        session.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_probability: 1.})

Iter:1280,Minibatch loss20484.523438,Training_accuracy:0.32031
Iter:2560,Minibatch loss7681.899414,Training_accuracy:0.59375
Iter:3840,Minibatch loss7614.326660,Training_accuracy:0.66406
Iter:5120,Minibatch loss6347.348633,Training_accuracy:0.75781
Iter:6400,Minibatch loss3328.585938,Training_accuracy:0.82031
Iter:7680,Minibatch loss3171.914551,Training_accuracy:0.82812
Iter:8960,Minibatch loss2867.481201,Training_accuracy:0.88281
Iter:10240,Minibatch loss1619.229736,Training_accuracy:0.86719
Iter:11520,Minibatch loss2337.446777,Training_accuracy:0.85156
Iter:12800,Minibatch loss2023.565552,Training_accuracy:0.90625
Iter:14080,Minibatch loss2743.434326,Training_accuracy:0.85156
Iter:15360,Minibatch loss2332.479248,Training_accuracy:0.89062
Iter:16640,Minibatch loss1516.997681,Training_accuracy:0.90625
Iter:17920,Minibatch loss2471.589355,Training_accuracy:0.89844
Iter:19200,Minibatch loss2848.899170,Training_accuracy:0.89844
Iter:20480,Minibatch loss1627.068604,Training_accuracy:0.8984